<a href="https://colab.research.google.com/github/Farnaaaz/TenniMatchPrediction/blob/main/TennisResultPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import numpy as np
import os
import pandas as pd
from io import StringIO
import io
import string
import random
import json
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from scipy.stats import norm
from google.colab import drive
import csv
from datetime import datetime

Filtering and Merging the TOURNAMENT data sets

In [53]:
tournaments1 = pd.read_csv('match_scores_1991-2016_unindexed_csv.csv')

In [54]:
#Choosing only data for year 2016
filtered_t1 = tournaments1[tournaments1['tourney_year_id'].str.startswith('2016-')]

In [55]:
filtered_t1.head()

,tourney_year_id,tourney_order,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,loser_seed,match_score_tiebreaks,winner_sets_won,loser_sets_won,winner_games_won,loser_games_won,winner_tiebreaks_won,loser_tiebreaks_won,match_id,match_stats_url_suffix
91415,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Finals,1,1,Milos Raonic,r975,milos-raonic,...,1,64 64,2,0,12,8,0,0,2016-339-r975-f324,/en/scores/2016/339/MS001/match-stats
91416,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Semi-Finals,2,1,Roger Federer,f324,roger-federer,...,8,61 64,2,0,12,5,0,0,2016-339-f324-tb69,/en/scores/2016/339/MS002/match-stats
91417,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Semi-Finals,2,2,Milos Raonic,r975,milos-raonic,...,7,76(5) 76(5),2,0,14,12,2,0,2016-339-r975-ta46,/en/scores/2016/339/MS003/match-stats
91418,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Quarter-Finals,3,1,Roger Federer,f324,roger-federer,...,NaN,64 67(4) 64,2,1,18,15,0,1,2016-339-f324-d875,/en/scores/2016/339/MS004/match-stats
91419,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Quarter-Finals,3,2,Bernard Tomic,ta46,bernard-tomic,...,2,63 16 63,2,1,13,12,0,0,2016-339-ta46-n552,/en/scores/2016/339/MS007/match-stats


In [56]:
filtered_t1_rows = filtered_t1.shape[0]

In [57]:
print(f"The filtered ATP1 DataFrame has {filtered_t1_rows} rows.")

The filtered ATP1 DataFrame has 3944 rows.


In [58]:
tournaments2 = pd.read_csv('match_scores_2017_unindexed_csv.csv')

In [59]:
t2_rows = tournaments2.shape[0]
print(f"The filtered ATP2 DataFrame has {t2_rows} rows.")

The filtered ATP2 DataFrame has 3830 rows.


In [60]:
#Joining two tables vertically
merged_tournaments = pd.concat([filtered_t1, tournaments2], ignore_index=True)

In [61]:
merged_tournaments.head()

,tourney_year_id,tourney_order,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,loser_seed,match_score_tiebreaks,winner_sets_won,loser_sets_won,winner_games_won,loser_games_won,winner_tiebreaks_won,loser_tiebreaks_won,match_id,match_stats_url_suffix
0,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Finals,1,1,Milos Raonic,r975,milos-raonic,...,1,64 64,2,0,12,8,0,0,2016-339-r975-f324,/en/scores/2016/339/MS001/match-stats
1,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Semi-Finals,2,1,Roger Federer,f324,roger-federer,...,8,61 64,2,0,12,5,0,0,2016-339-f324-tb69,/en/scores/2016/339/MS002/match-stats
2,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Semi-Finals,2,2,Milos Raonic,r975,milos-raonic,...,7,76(5) 76(5),2,0,14,12,2,0,2016-339-r975-ta46,/en/scores/2016/339/MS003/match-stats
3,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Quarter-Finals,3,1,Roger Federer,f324,roger-federer,...,NaN,64 67(4) 64,2,1,18,15,0,1,2016-339-f324-d875,/en/scores/2016/339/MS004/match-stats
4,2016-339,1,brisbane,/en/scores/archive/brisbane/339/2016/results,Quarter-Finals,3,2,Bernard Tomic,ta46,bernard-tomic,...,2,63 16 63,2,1,13,12,0,0,2016-339-ta46-n552,/en/scores/2016/339/MS007/match-stats


In [62]:
#verifying the merge, should be 3830 + 3944
tournament_rows = merged_tournaments.shape[0]
print(f"The filtered merged_ATP DataFrame has {tournament_rows} rows.")

The filtered merged_ATP DataFrame has 7774 rows.


Filtering and Merging the MATCH data sets

In [63]:
matches1 = pd.read_csv('match_stats_2017_unindexed_csv (1).csv')
matches2 = pd.read_csv('match_stats_1991-2016_unindexed_csv.csv')

In [64]:
#Choosing only data for year 2016
filtered_match2 = matches2[matches2['match_id'].str.startswith('2016-')]

In [65]:
filtered_match2.head()

,tourney_order,match_id,match_stats_url_suffix,match_time,match_duration,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,...,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_service_games_played,loser_return_games_played,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
88033,0,2016-339-f324-tb69,/en/scores/2016/339/MS002/match-stats,01:01:00,61.0,6.0,0.0,27.0,49.0,23.0,...,10.0,22.0,1.0,1.0,8.0,9.0,14.0,49.0,41.0,104.0
88034,0,2016-339-r975-f324,/en/scores/2016/339/MS001/match-stats,01:27:00,87.0,6.0,6.0,34.0,60.0,28.0,...,12.0,26.0,0.0,1.0,10.0,10.0,18.0,60.0,57.0,121.0
88035,0,2016-339-f324-d875,/en/scores/2016/339/MS004/match-stats,02:08:00,128.0,7.0,1.0,52.0,83.0,45.0,...,12.0,31.0,1.0,1.0,16.0,16.0,19.0,83.0,86.0,187.0
88036,0,2016-339-r975-ta46,/en/scores/2016/339/MS003/match-stats,01:48:00,108.0,13.0,3.0,49.0,79.0,40.0,...,11.0,30.0,0.0,4.0,12.0,12.0,20.0,79.0,77.0,163.0
88037,0,2016-339-ta46-n552,/en/scores/2016/339/MS007/match-stats,02:00:00,120.0,14.0,1.0,62.0,97.0,45.0,...,22.0,35.0,3.0,11.0,12.0,13.0,39.0,97.0,89.0,181.0


In [66]:
match2_rows = filtered_match2.shape[0]
print(f"The filtered merged_ATP DataFrame has {match2_rows} rows.")

The filtered merged_ATP DataFrame has 3923 rows.


In [67]:
match1_rows = matches1.shape[0]
print(f"The filtered ATP2 DataFrame has {match1_rows} rows.")

The filtered ATP2 DataFrame has 3811 rows.


In [68]:
#Joining two tables vertically
merged_matches = pd.concat([filtered_match2, matches1], ignore_index=True)

In [69]:
#verifying the merge, should be 3923 + 3811
matches_rows = merged_matches.shape[0]
print(f"The filtered merged_ATP DataFrame has {matches_rows} rows.")

The filtered merged_ATP DataFrame has 7734 rows.


Merging Tournaments and Matches tables with Players table

In [70]:
players = pd.read_csv('player_overviews_unindexed_csv (1).csv')

In [71]:
## Merge 'merged_matches' and 'merged_tournaments' on 'tourney_order'
merged_df = pd.merge(merged_matches, merged_tournaments, on='tourney_order', how='inner')

In [72]:
# Merge 'merged_df' with 'player' on 'winner_player_id' and update column names for winner
merged_df = pd.merge(merged_df, players, left_on='winner_player_id', right_on='player_id', how='left', suffixes=('', '_winner'))
merged_df.columns = [col if col not in players.columns else f"winner_{col}" for col in merged_df.columns]

# Merge 'merged_df' with 'player' on 'loser_player_id' and update column names for loser
merged_df = pd.merge(merged_df, players, left_on='loser_player_id', right_on='player_id', how='left', suffixes=('', '_loser'))
merged_df.columns = [col if col not in players.columns else f"loser_{col}" for col in merged_df.columns]

In [73]:
# Drop duplicate columns (e.g., 'player_id_x', 'player_id_y')
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

# Display the resulting DataFrame
merged_df.head()

,tourney_order,match_id_x,match_stats_url_suffix_x,match_time,match_duration,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,...,loser_birth_month,loser_birth_day,loser_turned_pro,loser_weight_lbs,loser_weight_kg,loser_height_ft,loser_height_inches,loser_height_cm,loser_handedness,loser_backhand
0,1,2016-891-cg80-bh09,/en/scores/2016/891/MS003/match-stats,02:57:00,177.0,17.0,1.0,92.0,134.0,66.0,...,8.0,8.0,1998.0,187.0,85.0,"6'1""",73.0,185.0,Right-Handed,One-Handed Backhand
1,1,2016-891-cg80-bh09,/en/scores/2016/891/MS003/match-stats,02:57:00,177.0,17.0,1.0,92.0,134.0,66.0,...,9.0,3.0,2011.0,180.0,82.0,"6'1""",73.0,185.0,Right-Handed,One-Handed Backhand
2,1,2016-891-cg80-bh09,/en/scores/2016/891/MS003/match-stats,02:57:00,177.0,17.0,1.0,92.0,134.0,66.0,...,10.0,21.0,2008.0,201.0,91.0,"6'5""",77.0,196.0,Right-Handed,Two-Handed Backhand
3,1,2016-891-cg80-bh09,/en/scores/2016/891/MS003/match-stats,02:57:00,177.0,17.0,1.0,92.0,134.0,66.0,...,5.0,16.0,2008.0,176.0,80.0,"6'3""",75.0,191.0,Right-Handed,One-Handed Backhand
4,1,2016-891-cg80-bh09,/en/scores/2016/891/MS003/match-stats,02:57:00,177.0,17.0,1.0,92.0,134.0,66.0,...,12.0,29.0,2007.0,165.0,75.0,"5'10""",70.0,178.0,Right-Handed,Two-Handed Backhand


Filtering features for ATP Dataset

In [74]:
selected_columns = ['tourney_year_id','tourney_slug','tourney_round_name','winner_name','winner_flag_code', 'winner_player_id', 'loser_name','loser_flag_code', 'loser_player_id','winner_sets_won','loser_sets_won', 'loser_birth_month', 'winner_birth_month', 'loser_weight_kg','loser_height_ft', 'winner_weight_kg',	'winner_height_ft']
ATP_features = merged_df[selected_columns]

# Display the result
ATP_features.head()

,tourney_year_id,tourney_slug,tourney_round_name,winner_name,winner_flag_code,winner_player_id,loser_name,loser_flag_code,loser_player_id,winner_sets_won,loser_sets_won,loser_birth_month,winner_birth_month,loser_weight_kg,loser_height_ft,winner_weight_kg,winner_height_ft
0,2016-339,brisbane,Finals,Milos Raonic,CAN,r975,Roger Federer,SUI,f324,2,0,8.0,12.0,85.0,"6'1""",98.0,"6'5"""
1,2016-339,brisbane,Semi-Finals,Roger Federer,SUI,f324,Dominic Thiem,AUT,tb69,2,0,9.0,8.0,82.0,"6'1""",85.0,"6'1"""
2,2016-339,brisbane,Semi-Finals,Milos Raonic,CAN,r975,Bernard Tomic,AUS,ta46,2,0,10.0,12.0,91.0,"6'5""",98.0,"6'5"""
3,2016-339,brisbane,Quarter-Finals,Roger Federer,SUI,f324,Grigor Dimitrov,BUL,d875,2,1,5.0,8.0,80.0,"6'3""",85.0,"6'1"""
4,2016-339,brisbane,Quarter-Finals,Bernard Tomic,AUS,ta46,Kei Nishikori,JPN,n552,2,1,12.0,10.0,75.0,"5'10""",91.0,"6'5"""


**Merging Betting dataset**

In [75]:
file_2017 = 'Betting/2017.xlsx'
file_2018 = 'Betting/2017.xlsx'
file_2019 = 'Betting/2017.xlsx'

# Read Excel files into DataFrames
df_2017 = pd.read_excel(file_2017)
df_2018 = pd.read_excel(file_2018)
df_2019 = pd.read_excel(file_2019)

In [76]:
# Remove specific columns from 2017 (not common)
columns_to_remove = ['EXW', 'EXL', 'LBW', 'LBL']

# Ensure df_2017 is defined with these columns
df_2017 = df_2017.drop(columns=columns_to_remove, errors='ignore')

col_order = ['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets', 'Comment', 'B365W', 'B365L', 'PSW', 'PSL', 'MaxW', 'MaxL', 'AvgW', 'AvgL']

# Reorder columns in each DataFrame
df_2017 = df_2017[col_order]
df_2018 = df_2018[col_order]
df_2019 = df_2019[col_order]

# Concatenate the DataFrames vertically
betting_merged = pd.concat([df_2017, df_2018, df_2019], ignore_index=True)

betting_merged.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,Lsets,Comment,B365W,B365L,PSW,PSL,MaxW,MaxL,AvgW,AvgL
0,1,Brisbane,Brisbane International,2017-01-01,ATP250,Outdoor,Hard,1st Round,3,Thompson J.,...,0.0,Completed,1.28,3.50,1.31,3.73,1.33,3.90,1.29,3.50
1,1,Brisbane,Brisbane International,2017-01-02,ATP250,Outdoor,Hard,1st Round,3,Mahut N.,...,1.0,Completed,1.50,2.50,1.58,2.52,1.59,2.55,1.54,2.43
2,1,Brisbane,Brisbane International,2017-01-02,ATP250,Outdoor,Hard,1st Round,3,Ferrer D.,...,0.0,Completed,1.90,1.80,2.10,1.81,2.15,1.82,2.01,1.77
3,1,Brisbane,Brisbane International,2017-01-02,ATP250,Outdoor,Hard,1st Round,3,Edmund K.,...,0.0,Completed,1.36,3.00,1.42,3.09,1.42,3.25,1.37,3.01
4,1,Brisbane,Brisbane International,2017-01-02,ATP250,Outdoor,Hard,1st Round,3,Dimitrov G.,...,0.0,Completed,1.40,2.75,1.41,3.13,1.46,3.13,1.41,2.85


Merging Kaggle Data

In [55]:
kaggle1 = pd.read_csv('Kaggle_tennis_atp/atp_matches_2019_updated.csv')
kaggle2 = pd.read_csv('Kaggle_tennis_atp/atp_matches_2020_updated.csv')

In [56]:
# Concatenate the DataFrames vertically
kaggle_merged = pd.concat([kaggle1, kaggle2], ignore_index=True)

kaggle_merged.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2019-M020,Brisbane,Hard,32,A,12/31/2018,300,105453,2.0,NaN,...,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0
1,2019-M020,Brisbane,Hard,32,A,12/31/2018,299,106421,4.0,NaN,...,52.0,36.0,7.0,10.0,10.0,13.0,16.0,1977.0,239.0,200.0
2,2019-M020,Brisbane,Hard,32,A,12/31/2018,298,105453,2.0,NaN,...,27.0,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0
3,2019-M020,Brisbane,Hard,32,A,12/31/2018,297,104542,NaN,PR,...,60.0,38.0,9.0,11.0,4.0,6.0,239.0,200.0,31.0,1298.0
4,2019-M020,Brisbane,Hard,32,A,12/31/2018,296,106421,4.0,NaN,...,56.0,46.0,19.0,15.0,2.0,4.0,16.0,1977.0,18.0,1855.0


Filtering features for Kaggle Dataset

In [57]:
selected_columns = ['tourney_id','tourney_name','surface','tourney_date','winner_id','winner_name','winner_hand','winner_ht','winner_ioc','winner_age','loser_id','loser_name','loser_hand','loser_ht','loser_ioc','loser_age','best_of','round','winner_rank','loser_rank']
kaggle_features = kaggle_merged[selected_columns]

# Display the result
kaggle_features.head()

,tourney_id,tourney_name,surface,tourney_date,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,best_of,round,winner_rank,loser_rank
0,2019-M020,Brisbane,Hard,12/31/2018,105453,Kei Nishikori,R,178.0,JPN,29.0,106421,Daniil Medvedev,R,198.0,RUS,22.8,3,F,9.0,16.0
1,2019-M020,Brisbane,Hard,12/31/2018,106421,Daniil Medvedev,R,198.0,RUS,22.8,104542,Jo-Wilfried Tsonga,R,188.0,FRA,33.7,3,SF,16.0,239.0
2,2019-M020,Brisbane,Hard,12/31/2018,105453,Kei Nishikori,R,178.0,JPN,29.0,104871,Jeremy Chardy,R,188.0,FRA,31.8,3,SF,9.0,40.0
3,2019-M020,Brisbane,Hard,12/31/2018,104542,Jo-Wilfried Tsonga,R,188.0,FRA,33.7,200282,Alex De Minaur,R,183.0,AUS,19.8,3,QF,239.0,31.0
4,2019-M020,Brisbane,Hard,12/31/2018,106421,Daniil Medvedev,R,198.0,RUS,22.8,105683,Milos Raonic,R,196.0,CAN,28.0,3,QF,16.0,18.0


Filetring features for Betting Dataset

In [79]:
selected_columns = ['ATP','Location','Tournament','Date','Series','Court','Surface','Round','Best of','Winner','Loser','WRank','LRank']
Betting_features = betting_merged[selected_columns]

# Display the result
Betting_features.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank
0,1,Brisbane,Brisbane International,2017-01-01,ATP250,Outdoor,Hard,1st Round,3,Thompson J.,Ymer E.,79.0,160.0
1,1,Brisbane,Brisbane International,2017-01-02,ATP250,Outdoor,Hard,1st Round,3,Mahut N.,Robert S.,39.0,54.0
2,1,Brisbane,Brisbane International,2017-01-02,ATP250,Outdoor,Hard,1st Round,3,Ferrer D.,Tomic B.,21.0,26.0
3,1,Brisbane,Brisbane International,2017-01-02,ATP250,Outdoor,Hard,1st Round,3,Edmund K.,Escobedo E.,45.0,141.0
4,1,Brisbane,Brisbane International,2017-01-02,ATP250,Outdoor,Hard,1st Round,3,Dimitrov G.,Johnson S.,17.0,33.0


Filtering Date for required tables

In [ ]:

# Open the  file for reading and writing
# For file atp_rankings_20s.csv

with open('Kaggle_tennis_atp\\atp_rankings_20s.csv', 'r', newline='') as infile, \
     open('Kaggle_tennis_atp\\atp_rankings_20s_updated.csv', 'w', newline='') as outfile:

    # Create a csv reader object
    reader = csv.reader(infile)

    # Create a csv writer object
    writer = csv.writer(outfile)

    # Read the header row
    headers = next(reader)

    # Write the headers to the output file
    writer.writerow(headers)

    # Iterate over the remaining rows
    for row in reader:
        # Convert the string in the first column to a datetime object
        date_str = row[0]
        date_obj = datetime.strptime(date_str, '%Y%m%d')

        # Format the datetime object as 'MM/DD/YYYY'
        formatted_date = date_obj.strftime('%m/%d/%Y')

        # Update the first column in the row with the formatted date
        row[0] = formatted_date

        # Write the updated row to the output file
        writer.writerow(row)

# For file atp_rankings_current.csv

with open('Kaggle_tennis_atp\\atp_rankings_current.csv', 'r', newline='') as infile, \
     open('Kaggle_tennis_atp\\atp_rankings_current_updated.csv', 'w', newline='') as outfile:

    # Create a csv reader object
    reader = csv.reader(infile)

    # Create a csv writer object
    writer = csv.writer(outfile)

    # Read the header row
    headers = next(reader)

    # Write the headers to the output file
    writer.writerow(headers)

    # Iterate over the remaining rows
    for row in reader:
        # Convert the string in the first column to a datetime object
        date_str = row[0]
        date_obj = datetime.strptime(date_str, '%Y%m%d')

        # Format the datetime object as 'MM/DD/YYYY'
        formatted_date = date_obj.strftime('%m/%d/%Y')

        # Update the first column in the row with the formatted date
        row[0] = formatted_date

        # Write the updated row to the output file
        writer.writerow(row)




In [ ]:
def update_date_format(input_file, output_file, date_column_index):
    # Open the input and output files
    with open(input_file, 'r', newline='') as infile, \
         open(output_file, 'w', newline='') as outfile:

        # Create a csv reader object
        reader = csv.reader(infile)

        # Create a csv writer object
        writer = csv.writer(outfile)

        # Read the header row
        headers = next(reader)

        # Write the headers to the output file
        writer.writerow(headers)

        # Iterate over the remaining rows
        for row in reader:
            # Convert the string in the specified date column to a datetime object
            date_str = row[date_column_index]
            date_obj = datetime.strptime(date_str, '%Y%m%d')

            # Format the datetime object as 'MM/DD/YYYY'
            formatted_date = date_obj.strftime('%m/%d/%Y')

            # Update the specified date column in the row with the formatted date
            row[date_column_index] = formatted_date

            # Write the updated row to the output file
            writer.writerow(row)


# Update the date format in 'atp_matches_2019.csv - 'atp_matches_2023.csv' files

update_date_format('Kaggle_tennis_atp\\atp_matches_2019.csv', 'Kaggle_tennis_atp\\atp_matches_2019_updated.csv', 5)
update_date_format('Kaggle_tennis_atp\\atp_matches_2020.csv', 'Kaggle_tennis_atp\\atp_matches_2020_updated.csv', 5)
update_date_format('Kaggle_tennis_atp\\atp_matches_2021.csv', 'Kaggle_tennis_atp\\atp_matches_2021_updated.csv', 5)
update_date_format('Kaggle_tennis_atp\\atp_matches_2022.csv', 'Kaggle_tennis_atp\\atp_matches_2022_updated.csv', 5)
update_date_format('Kaggle_tennis_atp\\atp_matches_2023.csv', 'Kaggle_tennis_atp\\atp_matches_2023_updated.csv', 5)

# Update the date format in 'atp_matches_futures_2019.csv - 'atp_matches_futures_2023.csv' files

update_date_format('Kaggle_tennis_atp\\atp_matches_futures_2019.csv', 'Kaggle_tennis_atp\\atp_matches_futures_2019_updated.csv', 5)
update_date_format('Kaggle_tennis_atp\\atp_matches_futures_2020.csv', 'Kaggle_tennis_atp\\atp_matches_futures_2020_updated.csv', 5)
update_date_format('Kaggle_tennis_atp\\atp_matches_futures_2021.csv', 'Kaggle_tennis_atp\\atp_matches_futures_2021_updated.csv', 5)
update_date_format('Kaggle_tennis_atp\\atp_matches_futures_2022.csv', 'Kaggle_tennis_atp\\atp_matches_futures_2022_updated.csv', 5)
update_date_format('Kaggle_tennis_atp\\atp_matches_futures_2023.csv', 'Kaggle_tennis_atp\\atp_matches_futures_2023_updated.csv', 5)

# Update the date format in 'atp_matches_qual_chall_2019.csv - 'atp_matches_qual_chall_2023.csv' files

update_date_format('Kaggle_tennis_atp\\atp_matches_qual_chall_2019.csv', 'Kaggle_tennis_atp\\atp_matches_qual_chall_2019_updated.csv', 5)
update_date_format('Kaggle_tennis_atp\\atp_matches_qual_chall_2020.csv', 'Kaggle_tennis_atp\\atp_matches_qual_chall_2020_updated.csv', 5)
update_date_format('Kaggle_tennis_atp\\atp_matches_qual_chall_2021.csv', 'Kaggle_tennis_atp\\atp_matches_qual_chall_2021_updated.csv', 5)
update_date_format('Kaggle_tennis_atp\\atp_matches_qual_chall_2022.csv', 'Kaggle_tennis_atp\\atp_matches_qual_chall_2022_updated.csv', 5)
update_date_format('Kaggle_tennis_atp\\atp_matches_qual_chall_2023.csv', 'Kaggle_tennis_atp\\atp_matches_qual_chall_2023_updated.csv', 5)